In [145]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from collections import deque
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

Y_IM_SCALAR = 0.6
LEFT_SLOPES = deque()
RIGHT_SLOPES = deque()
SLOPE_TOLERANCE = 0.2
AVG_COUNT = 30

def process_image(image):
    """Gray Scale"""
    gray = grayscale(image)
    """Gaussian Blurred"""
    kernel_size = 5
    g_smooth = gaussian_blur(image, kernel_size)
    """Apply Canny"""
    low_threshold = 50
    high_threshold = 150
    cannyEdges = canny(g_smooth, low_threshold, high_threshold)

    """Define vertices and mask"""
    imshape = image.shape
    vertices = np.array([[(0 + imshape[1]*0.08,imshape[0]),(imshape[1] * 0.44, imshape[0] * Y_IM_SCALAR), (imshape[1] - imshape[1]*0.47, imshape[0] * Y_IM_SCALAR), (imshape[1] - imshape[1]* 0.08,imshape[0])]], dtype=np.int32)
    masked = region_of_interest(cannyEdges, vertices)

    """apply Hough"""
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    lines = hough_lines(masked, rho, theta, threshold, min_line_length, max_line_gap)
    """Return comoposited image"""
    return weighted_img(lines, image)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def find_new_endpoints(lines_left_right, imshape):
    left_segment = lines_left_right[0][0]
    right_segment = lines_left_right[1][0]
    
    return [find_endpoints_from(left_segment, imshape), find_endpoints_from(right_segment, imshape, False)]

def find_endpoints_from(segment, imshape, left=True):
    mid = midpoint(segment)
    s = slope(segment)
    if s == 0:
        return [[0,0,0,0]]
    
    if left:
        s = select_slope(s, LEFT_SLOPES)
    else:
        s = select_slope(s, RIGHT_SLOPES)
    
    """y = mx + b"""
    constant = mid[1] - s * mid[0]
    """new segment is where y = max, y = min of region, solve for x"""
    """x = (y - b)/m"""
    y1 = (imshape[0] * Y_IM_SCALAR)
    x1 = (y1 - constant)/s
    y2 = imshape[0]
    x2 = (y2 - constant)/s
    
    return [[int(x1), int(y1), int(x2), int(y2)]]

def select_slope(new_slope, slopes):
    if len(slopes) == 0:
        return new_slope
    
    mean = np.mean(slopes)
    dif = abs(mean - new_slope)/ (mean + new_slope)/2
    
    if dif > SLOPE_TOLERANCE:
        return mean
    else:
        if len(slopes) < AVG_COUNT:
            slopes.append(new_slope)
            return np.mean(slopes)
        else:
            slopes.popleft()
            slopes.append(new_slope)
            return np.mean(slopes)
            
def draw_lines_V2(img, lines, color=[255,0,0], thickness=12):
    left_right_min_max = find_min_max_left_right(lines, img.shape)
    newLines = find_new_endpoints(left_right_min_max, img.shape)
    
    for line in newLines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1,y1), (x2,y2), color, thickness)

def midpoint(endpoints):
    return [(endpoints[0] + endpoints[2])/2, (endpoints[1] + endpoints[3])/2]
    
"""imshape[0]= y imshape[1]= x for some reason"""
def find_min_max_left_right(lines, imshape):
    minLeft = [0, imshape[0]]
    minRight = [0, imshape[0]]
    maxLeft = [0,0]
    maxRight = [0,0]  
    for line in lines:
        slope = find_slope(line)
        """Only have to compare the Y coor"""
        for x1,y1,x2,y2 in line: 
            if slope < 0:
                if y1 < minLeft[1]:
                    minLeft = [x1, y1]
                if y2 < minLeft[1]:
                    minLeft = [x2, y2]
                if y1 > maxLeft[1]:
                    maxLeft = [x1, y1]
                if y2 > maxLeft[1]:
                    maxLeft = [x2, y2]
            else:
                if y1 < minRight[1]:
                    minRight = [x1, y1]
                if y2 < minRight[1]:
                    minRight = [x2, y2]
                if y1 > maxRight[1]:
                    maxRight = [x1, y1]
                if y2 > maxRight[1]:
                    maxRight = [x2, y2]
    
    return [[[minLeft[0], minLeft[1], maxLeft[0], maxLeft[1]]], [[minRight[0], minRight[1], maxRight[0], maxRight[1]]]]

def find_slope(line):
    for x1,y1,x2,y2 in line:
        slope = (y2 - y1)/(x2 - x1)
    return slope

def slope(line):
    if line[2] - line[0] == 0:
        return 0
    slope = (line[3] - line[1])/(line[2] - line[0])
    return slope

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines_V2(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

# image = mpimg.imread('test_images/solidYellowCurve2.jpg')

# result = process_image(image)

# plt.imshow(result)

white_output = 'test_videos_output/solidYellowLeft.mp4'

clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)




[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4



100%|█████████▉| 681/682 [00:10<00:00, 67.58it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 13.6 s, sys: 3.5 s, total: 17.1 s
Wall time: 10.5 s
